In [ ]:
title: "bayescan_v1"
author: "Caroline Rzucidlo"
date: "2021"
output: html_document
---
  
# this script utilizes the bayescan plotR.r script. it can be found here: http://www.cmpg.unibe.ch/software/BayeScan/files/BayeScan2.0_manual.pdf

library(boa)
library(dplyr)

# set the working directory
setwd("~/Documents/Environmental Bioinformatics/Final Project/R/")

# load the bayescan script that comes with the package
source("~/Documents/Environmental Bioinformatics/Final Project/R/plot_R.r")

############################################################
##### THIS VERSION ACCOUNTS FOR HOST IDENTITY IN POPULATION INFORMATION
############################################################

host <- plot_bayescan("../output/seq_outliers/host_fst.txt", 1, FDR=0.05)
summary(host)

# write a table to play with data later
# make this data have all of the STATS
h <- read.table("../output/seq_outliers/host_fst.txt")
h$ID <- seq.int(nrow(h))
colnames(h) <- c("BAYES_prob", "BAYES_log10.PO.", "BAYES_qval", "BAYES_alpha", "BAYES_fst", "ID")

# make host outlier dataframe
hout <- as.data.frame(host$outliers)
colnames(hout) <- c("ID")
hostouts <- semi_join(h, hout, by = "ID", copy = FALSE)
sel <- read.table("../output/seq_outliers/host.sel", colClasses = "numeric")
parameter="Fst1"
plot(density(sel[[parameter]]), xlab=parameter, main=paste(parameter, "posterior distribution host identity"))
boa.hpd(sel[[parameter]], 0.05)

# write file of all stats
write.table(h, "../output/seq_outliers/host_bayes_stats.txt", sep="\t")

############################################################
##### THIS VERSION ACCOUNTS FOR LOCATION IN POPULATION INFORMATION
############################################################

location <- plot_bayescan("../output/seq_outliers/loc_fst.txt", 1, FDR=0.05)
summary(location)

# write a table to play with data later
# make this data have all of the STATS
l <- read.table("../output/seq_outliers/loc_fst.txt")
l$ID <- seq.int(nrow(l))
colnames(l) <- c("BAYES_prob", "BAYES_log10.PO.", "BAYES_qval", "BAYES_alpha", "BAYES_fst", "ID")

# make host outlier dataframe
lout <- as.data.frame(location$outliers)
colnames(lout) <- c("ID")
locouts <- semi_join(l, lout, by = "ID", copy = FALSE)

# write file of all stats
write.table(l, "../output/seq_outliers/location_bayes_stats.txt", sep="\t")
sel <- read.table("../output/seq_outliers/loc.sel", colClasses = "numeric")
parameter="Fst1"
plot(density(sel[[parameter]]), xlab=parameter, main=paste(parameter, "posterior distribution - location"))
boa.hpd(sel[[parameter]], 0.05)

############################################################
##### THIS VERSION ACCOUNTS FOR HOST AND LOCATION IN POPULATION INFORMATION
############################################################

Hloc <- plot_bayescan("../output/seq_outliers/hostXloc_fst.txt", 1, FDR=0.05)
summary(Hloc)

# write a table to play with data later
# make this data have all of the STATS
HL <- read.table("../output/seq_outliers/hostXloc_fst.txt")
HL$ID <- seq.int(nrow(HL))
colnames(HL) <- c("BAYES_prob", "BAYES_log10.PO.", "BAYES_qval", "BAYES_alpha", "BAYES_fst", "ID")

# make host outlier dataframe
HLout <- as.data.frame(Hloc$outliers)
colnames(HLout) <- c("ID")
HLocouts <- semi_join(HL, HLout, by = "ID", copy = FALSE)

# write file of all stats
write.table(HL, "../output/seq_outliers/hostXloc_bayes_stats.txt", sep="\t")
sel <- read.table("../output/seq_outliers/hostXloc.sel", colClasses = "numeric")
parameter="Fst1"
plot(density(sel[[parameter]]), xlab=parameter, main=paste(parameter, "posterior distribution - host x loc"))
boa.hpd(sel[[parameter]], 0.05)

###################
###########
# see if there's overlap between the different bayescan runs
hxlhost <- merge(HLocouts, hostouts, by = "ID")

# label them as host_and_hostXloc
hxlhost["version_BAYES"] <- "host_and_hostXloc"
hxlloc <- merge(HLocouts, locouts, by = "ID")

# label them as loc_and_hostXloc
hxlloc["version_BAYES"] <- "loc_and_hostXloc"

# rbind the overlap for both versions
overlap <- rbind(hxlhost, hxlloc)
count(overlap)

### find the excluded ones
# did this by subsetting ID. resource: https://stackoverflow.com/questions/5812478/how-i-can-select-rows-from-a-dataframe-that-do-not-match
locsub <- subset(locouts, !(locouts$ID %in% HLocouts$ID))

# count to check
count(locsub)
hostsub <- subset(hostouts, !(hostouts$ID %in% HLocouts$ID))

#count to check
count(hostsub)

## combine host and loc outlier tables 
# first add host/loc column to dataframes
locsub["version_BAYES"] <- "location"
hostsub["version_BAYES"] <- "host"

# concatenate subsets using rbind
sub <- rbind(locsub, hostsub)

### concatenate the overlaps and subsetted samples
# make them into dataframes first
# the bind_rows() / dplyr function is a lifesaver!!!
sub <- as.data.frame(sub)
overlap <- as.data.frame(overlap)
all <- bind_rows(sub, overlap)
count(all)

# write out the data
#write.table(all, "alloutlier_BAYESCAN_stats.txt", sep = "\t")
#### the old bayescan table was super ugly. writing a new one
#just rbind the loc, host, and hostXloc versions (anything to have a zillion columns for the bayes output)
# the objects are hostouts and locouts
# all of the hostXloc outliers overlapped with the host and loc outliers, leaving them alone because it makes the merged files very very messy
hostouts$Outlier_Type <- c("Bayescan Host")
locouts$Outlier_Type <- c("Bayescan Location")
both <- rbind(hostouts,locouts)
write.table(both, "../output/seq_outliers/bayescan_outliers_clean.txt", sep = "\t")
```